###### word2vec의 구현 1단계: 모델을 신경망으로 구축하기.
###### 여기서는 CBOW(Continuous bag-of-words) 사용.

# 3-2-1. CBOW 모델의 추론 처리

 - CBOW는 주변 맥락으로부터 Target을 추측하는 용도의 신경망.
 - 맥락을 CBOW모델에 입력하기 위해, 맥락을 원핫 표현으로 변환.

In [4]:
#파이썬으로 구현한 CBOW 모델의 추론 처리
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

# sample text: you say goodbye and I say hello.

#샘플 맥락 데이터(여기서는 context를 2개로 가정한다.)
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])  #you
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])  #goodbye

#가중치 초기화
W_in = np.random.randn(7, 3)  # 입력층 -> 은닉층 가중치
W_out = np.random.randn(3, 7) # 은닉층 -> 출력층 가중치

#계층 생성: 입력층은 맥락 수(2개)만큼 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

#순전파
h0 = in_layer0.forward(c0) #.forward() : 순전파
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)   #각 단어의 점수

[[-0.59242689 -1.81777007  1.65108843 -1.14086699  0.44555298 -1.66469823
  -0.45125403]]


# 3-2-2. CBOW 모델의 학습

 - 올바른 예측을 할 수 있도록 "가중치 조정"이 이루어짐.
 - 가중치 조정의 결과, 가중치 W_in, W_out 모두에 단어의 출현 패턴을 파악한 벡터가 학습됨.
 
 - CBOW 모델은 단어 출현 패턴을 학습 시 사용한 코퍼스로부터 배움. 따라서 말뭉치가 다르면 얻어지는 단어의 분산 표현도 다름.

 - 앞에서 구현한 신경망에 Softmax, Cross-Entropy-Error 계층을 추가하여 손실(loss)을 구한 다음, 이를 학습에 이용.
 - Softmax + Cross-Entropy-Error를 합쳐 Softmax with Loss라는 하나의 계층으로 구현!

# 3-2-3. word2vec의 가중치와 분산 표현

 - word2vec에서 사용되는 신경망의 가중치 종류: 입력 측 완전연결계층의 가중치(W_in), 출력 측 완전연결계층의 가중치(W_out)
 - W_in의 각 행은 각 단어의 분산 표현, W_out은 단어의 의미가 인코딩된 벡터들.
 - W_out에서는 각 단어의 분산 표현이 열 방향(수직 방향)으로 저장된다.
 - 즉, W_in과 W_out 모두에서 각 단어의 분산표현을 확인할 수 있다.
 - 그러면 최종적으로 분산 표현으로는 어떤 가중치를 선택하는 것이 좋을까?
    * 대부분 '입력 층의 가중치(W_in)'만 이용한다.